In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("philosopher0808/real-vs-ai-generated-faces-dataset")

# print("Path to dataset files:", path)

In [2]:
import os

base_path = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1"

print(os.listdir(base_path))


['dataset', 'data_source']


In [3]:
from torchvision import transforms

IMG_SIZE = 128

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),  # solo en train
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])


In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

DATASET_PATH = r"C:\Users\Usuario\.cache\kagglehub\datasets\philosopher0808\real-vs-ai-generated-faces-dataset\versions\1\dataset\dataset"

train_dataset = ImageFolder(
    root=f"{DATASET_PATH}/train",
    transform=transform
)

val_dataset = ImageFolder(
    root=f"{DATASET_PATH}/validate",
    transform=transform
)

test_dataset = ImageFolder(
    root=f"{DATASET_PATH}/test",
    transform=transform
)


In [5]:
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FaceClassifierCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 1)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 128 → 64
        x = self.pool(F.relu(self.conv2(x)))  # 64 → 32
        x = self.pool(F.relu(self.conv3(x)))  # 32 → 16

        x = x.view(x.size(0), -1)              # Flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = torch.sigmoid(self.fc2(x))

        return x


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FaceClassifierCNN().to(device)


In [8]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [9]:
def train_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [10]:
def eval_epoch(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = (outputs > 0.5).int().squeeze()

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total


In [11]:
EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss = train_epoch(model, train_loader)
    val_acc = eval_epoch(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f}")


Epoch 1/10 | Loss: 0.5281 | Val Acc: 0.7718
Epoch 2/10 | Loss: 0.4439 | Val Acc: 0.8047
Epoch 3/10 | Loss: 0.3889 | Val Acc: 0.8432


KeyboardInterrupt: 

In [ ]:
test_acc = eval_epoch(model, test_loader)
print("Test Accuracy:", test_acc)
